In [16]:
# import libs
import os
import pandas as pd

In [55]:
print(os.listdir('../JUNO/Datasets'))

['train_lpmt_hits.h5', 'spmt_pos.csv', '.DS_Store', 'train_lpmt_n_hits.csv', 'train_spmt_n_hits.csv', 'train_info.csv', 'train_spmt_hits.h5', 'lpmt_pos.csv']


In [56]:
Datasets = dict()
for file in os.listdir("../JUNO/Datasets"):
    if file.split('.')[1] == 'csv':
        Datasets[file.split('.')[0]] = pd.read_csv('../JUNO/Datasets/' + str(file))
    elif file.split('.')[1] == 'h5':
        Datasets[file.split('.')[0]] = pd.read_hdf('../JUNO/Datasets/' + str(file), mode='r')
        
    